# 0. Install and Import Dependencies

In [1]:
# !pip install mediapipe opencv-python

In [2]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
import matplotlib.pyplot as plt

import tensorflow as tf
import pandas as pd
import pickle

# 2. Determining Joints

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

# 3. Calculate Angles

In [3]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

# 4. Curl Counter

In [4]:
def compute_points(df,landmarks) :
    new_entry = {'NOSE_X' : landmarks[mp_pose.PoseLandmark.NOSE.value].x, 
                                                    'NOSE_Y' : landmarks[mp_pose.PoseLandmark.NOSE.value].y,
                                                    'NOSE_Z' : landmarks[mp_pose.PoseLandmark.NOSE.value].z,
                                                    'LEFT_SHOULDER_X' : landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                                    'LEFT_SHOULDER_Y' : landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
                                                    'LEFT_SHOULDER_Z' : landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z,
                                                    'RIGHT_SHOULDER_X' : landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                                    'RIGHT_SHOULDER_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y,
                                                    'RIGHT_SHOULDER_Z' : landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z,
                                                    'LEFT_ELBOW_X' : landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                                                    'LEFT_ELBOW_Y' : landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y,
                                                    'LEFT_ELBOW_Z' : landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].z,
                                                    'RIGHT_ELBOW_X' : landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                                                    'RIGHT_ELBOW_Y': landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y,
                                                    'RIGHT_ELBOW_Z': landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].z,
                                                    'LEFT_WRIST_X' : landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                                                    'LEFT_WRIST_Y' : landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y,
                                                    'LEFT_WRIST_Z' : landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].z,
                                                    'RIGHT_WRIST_X' : landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                                                    'RIGHT_WRIST_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y,
                                                    'RIGHT_WRIST_Z' : landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].z,
                                                    'LEFT_PINKY_X' : landmarks[mp_pose.PoseLandmark.LEFT_PINKY.value].x,
                                                    'LEFT_PINKY_Y' : landmarks[mp_pose.PoseLandmark.LEFT_PINKY.value].y,
                                                    'LEFT_PINKY_Z' : landmarks[mp_pose.PoseLandmark.LEFT_PINKY.value].z,
                                                    'RIGHT_PINKY_X' : landmarks[mp_pose.PoseLandmark.RIGHT_PINKY.value].x,
                                                    'RIGHT_PINKY_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_PINKY.value].y,
                                                    'RIGHT_PINKY_Z' : landmarks[mp_pose.PoseLandmark.RIGHT_PINKY.value].z,
                                                    'LEFT_INDEX_X' : landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].x,
                                                    'LEFT_INDEX_Y': landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].y,
                                                    'LEFT_INDEX_Z': landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].z,
                                                    'RIGHT_INDEX_X' : landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].x,
                                                    'RIGHT_INDEX_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].y,
                                                    'RIGHT_INDEX_Z' : landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].z,
                                                    'LEFT_THUMB_X' : landmarks[mp_pose.PoseLandmark.LEFT_THUMB.value].x,
                                                    'LEFT_THUMB_Y' : landmarks[mp_pose.PoseLandmark.LEFT_THUMB.value].y,
                                                    'LEFT_THUMB_Z' : landmarks[mp_pose.PoseLandmark.LEFT_THUMB.value].z,
                                                    'RIGHT_THUMB_X' : landmarks[mp_pose.PoseLandmark.RIGHT_THUMB.value].x,
                                                    'RIGHT_THUMB_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_THUMB.value].y,
                                                    'RIGHT_THUMB_Z' : landmarks[mp_pose.PoseLandmark.RIGHT_THUMB.value].z,
                                                    'LEFT_HIP_X' : landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                                                    'LEFT_HIP_Y' : landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y,
                                                    'LEFT_HIP_Z' : landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z,
                                                    'RIGHT_HIP_X' : landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                                                    'RIGHT_HIP_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y,
                                                    'RIGHT_HIP_Z' : landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z,
                                                    'LEFT_KNEE_X' : landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                                                    'LEFT_KNEE_Y' : landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y,
                                                    'LEFT_KNEE_Z' : landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].z,
                                                    'RIGHT_KNEE_X' : landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                                                    'RIGHT_KNEE_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y,
                                                    'RIGHT_KNEE_Z' : landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].z,
                                                    'LEFT_ANKLE_X' : landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                                                    'LEFT_ANKLE_Y' : landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y,
                                                    'LEFT_ANKLE_Z' : landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].z,
                                                    'RIGHT_ANKLE_X' : landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                                                    'RIGHT_ANKLE_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y,
                                                    'RIGHT_ANKLE_Z' : landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].z,
                                                    'LEFT_HEEL_X' : landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].x,
                                                    'LEFT_HEEL_Y' : landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y,
                                                    'LEFT_HEEL_Z' : landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].z,
                                                    'RIGHT_HEEL_X' : landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].x, 
                                                    'RIGHT_HEEL_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].y,
                                                    'RIGHT_HEEL_Z' : landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].z,
                                                    'LEFT_FOOT_INDEX_X' : landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,  
                                                    'LEFT_FOOT_INDEX_Y' : landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y,  
                                                    'LEFT_FOOT_INDEX_Z' : landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].z,  
                                                    'RIGHT_FOOT_INDEX_X' : landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x,
                                                    'RIGHT_FOOT_INDEX_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y,
                                                    'RIGHT_FOOT_INDEX_Z' : landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].z}
    return df.append(new_entry,ignore_index = True)

def create_empty_df():
    return pd.DataFrame(columns =  ['NOSE_X',
 'NOSE_Y',
 'NOSE_Z',
 'LEFT_SHOULDER_X',
 'LEFT_SHOULDER_Y',
 'LEFT_SHOULDER_Z',
 'RIGHT_SHOULDER_X',
 'RIGHT_SHOULDER_Y',
 'RIGHT_SHOULDER_Z',
 'LEFT_ELBOW_X',
 'LEFT_ELBOW_Y',
 'LEFT_ELBOW_Z',
 'RIGHT_ELBOW_X',
 'RIGHT_ELBOW_Y',
 'RIGHT_ELBOW_Z',
 'LEFT_WRIST_X',
 'LEFT_WRIST_Y',
 'LEFT_WRIST_Z',
 'RIGHT_WRIST_X',
 'RIGHT_WRIST_Y',
 'RIGHT_WRIST_Z',
 'LEFT_PINKY_X',
 'LEFT_PINKY_Y',
 'LEFT_PINKY_Z',
 'RIGHT_PINKY_X',
 'RIGHT_PINKY_Y',
 'RIGHT_PINKY_Z',
 'LEFT_INDEX_X',
 'LEFT_INDEX_Y',
 'LEFT_INDEX_Z',
 'RIGHT_INDEX_X',
 'RIGHT_INDEX_Y',
 'RIGHT_INDEX_Z',
 'LEFT_THUMB_X',
 'LEFT_THUMB_Y',
 'LEFT_THUMB_Z',
 'RIGHT_THUMB_X',
 'RIGHT_THUMB_Y',
 'RIGHT_THUMB_Z',
 'LEFT_HIP_X',
 'LEFT_HIP_Y',
 'LEFT_HIP_Z',
 'RIGHT_HIP_X',
 'RIGHT_HIP_Y',
 'RIGHT_HIP_Z',
 'LEFT_KNEE_X',
 'LEFT_KNEE_Y',
 'LEFT_KNEE_Z',
 'RIGHT_KNEE_X',
 'RIGHT_KNEE_Y',
 'RIGHT_KNEE_Z',
 'LEFT_ANKLE_X',
 'LEFT_ANKLE_Y',
 'LEFT_ANKLE_Z',
 'RIGHT_ANKLE_X',
 'RIGHT_ANKLE_Y',
 'RIGHT_ANKLE_Z',
 'LEFT_HEEL_X',
 'LEFT_HEEL_Y',
 'LEFT_HEEL_Z',
 'RIGHT_HEEL_X',
 'RIGHT_HEEL_Y',
 'RIGHT_HEEL_Z',
 'LEFT_FOOT_INDEX_X',
 'LEFT_FOOT_INDEX_Y',
 'LEFT_FOOT_INDEX_Y',
 'RIGHT_FOOT_INDEX_X',
 'RIGHT_FOOT_INDEX_Y',
 'RIGHT_FOOT_INDEX_Z'])

In [ ]:
X_cols = ['NOSE_X',
 'NOSE_Y',
 'NOSE_Z',
 'LEFT_SHOULDER_X',
 'LEFT_SHOULDER_Y',
 'LEFT_SHOULDER_Z',
 'RIGHT_SHOULDER_X',
 'RIGHT_SHOULDER_Y',
 'RIGHT_SHOULDER_Z',
 'LEFT_ELBOW_X',
 'LEFT_ELBOW_Y',
 'LEFT_ELBOW_Z',
 'RIGHT_ELBOW_X',
 'RIGHT_ELBOW_Y',
 'RIGHT_ELBOW_Z',
 'LEFT_WRIST_X',
 'LEFT_WRIST_Y',
 'LEFT_WRIST_Z',
 'RIGHT_WRIST_X',
 'RIGHT_WRIST_Y',
 'RIGHT_WRIST_Z',
 'LEFT_PINKY_X',
 'LEFT_PINKY_Y',
 'LEFT_PINKY_Z',
 'RIGHT_PINKY_X',
 'RIGHT_PINKY_Y',
 'RIGHT_PINKY_Z',
 'LEFT_INDEX_X',
 'LEFT_INDEX_Y',
 'LEFT_INDEX_Z',
 'RIGHT_INDEX_X',
 'RIGHT_INDEX_Y',
 'RIGHT_INDEX_Z',
 'LEFT_THUMB_X',
 'LEFT_THUMB_Y',
 'LEFT_THUMB_Z',
 'RIGHT_THUMB_X',
 'RIGHT_THUMB_Y',
 'RIGHT_THUMB_Z',
 'LEFT_HIP_X',
 'LEFT_HIP_Y',
 'LEFT_HIP_Z',
 'RIGHT_HIP_X',
 'RIGHT_HIP_Y',
 'RIGHT_HIP_Z',
 'LEFT_KNEE_X',
 'LEFT_KNEE_Y',
 'LEFT_KNEE_Z',
 'RIGHT_KNEE_X',
 'RIGHT_KNEE_Y',
 'RIGHT_KNEE_Z',
 'LEFT_ANKLE_X',
 'LEFT_ANKLE_Y',
 'LEFT_ANKLE_Z',
 'RIGHT_ANKLE_X',
 'RIGHT_ANKLE_Y',
 'RIGHT_ANKLE_Z',
 'LEFT_HEEL_X',
 'LEFT_HEEL_Y',
 'LEFT_HEEL_Z',
 'RIGHT_HEEL_X',
 'RIGHT_HEEL_Y',
 'RIGHT_HEEL_Z',
 'LEFT_FOOT_INDEX_X',
 'LEFT_FOOT_INDEX_Y',
 'LEFT_FOOT_INDEX_Y',
 'RIGHT_FOOT_INDEX_X',
 'RIGHT_FOOT_INDEX_Y',
 'RIGHT_FOOT_INDEX_Z']


colX = []
colY = []
colZ = []

for col in X_cols :
     if 'X' in col:
          colX.append(col)
     elif 'Y' in col:
          colY.append(col)
     elif 'Z' in col:
          colZ.append(col)

In [7]:
cap = cv2.VideoCapture(0)

# Curl counter variables
counter = 0 
stage = None

# Exercice classification 
exercice = 'No exercice'

df = create_empty_df()
model = tf.keras.models.load_model('./Model/model2')

with open('./Model/encoder2.pkl','rb') as file:
    encode = pickle.load(file)

step = 1

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:

            landmarks = results.pose_landmarks.landmark
            df = compute_points(df,landmarks)


            if step%12 == 0 : #each 0.5 sec
                X_train_X = df[colX].apply(lambda x : x - np.mean(x),axis=1)
                X_train_Y = df[colY].apply(lambda y : y - np.mean(y),axis=1)
                X_train_Z = df[colZ].apply(lambda z : z - np.mean(z),axis=1)
                X = pd.concat([X_train_X,X_train_Y,X_train_Z],axis=1)
                predictions = encode.inverse_transform(np.argmax(model.predict(X),axis=1))
                values, counts = np.unique(predictions, return_counts=True)
                df = create_empty_df()
                exercice = values[np.argmax(counts)]
                print(exercice)
            step += 1





                       
        except:
            pass
        
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Rep data

        cv2.putText(image, exercice, (20,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)

        
        # Stage data
        cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

pushup
pushup
pushup
pushup
pushup
pushup
pushup
pushup
pushup
pushup
pushup
pushup
pushup
pushup
pushup
pushup
pushup
pushup
squats
squats
squats
squats
squats
squats
squats
squats
squats
squats
squats
squats
pushup
pushup
pushup
pushup
pushup
pushup
pushup
pushup
pushup
pushup
pushup
pushup
pushup
pushup
squats
situp
squats
situp
squats
squats
squats
situp
squats
squats
squats
squats
squats
squats
squats
squats
squats
squats
squats
squats
squats
squats
squats
squats
squats
squats
situp
situp
squats
squats
situp
squats
squats
situp
squats
squats
situp
situp
squats
situp
situp
situp
squats
squats
squats
squats
squats
squats
squats
squats
squats
squats
squats
squats
situp
situp
pushup
pushup
situp
situp
pushup
squats
squats
squats
situp
situp
situp
situp
squats
pushup
pushup
squats
squats
squats
squats
dips
squats
squats
squats
squats
squats
squats
situp
pushup
pushup
pushup
squats
squats
pushup
pushup
pushup
pushup
pushup
pushup
pushup
pushup
pushup
pushup
pushup
pushup
pushup
pushup
p